In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [2]:
# read data
birth = pd.read_csv('birth.csv')
edu = pd.read_csv('edu.csv')
spouse_edu = pd.read_csv('spouse_edu.csv')
marstat = pd.read_csv('marstat.csv')

In [3]:
birth.columns = ['id', 'birth_year', 'gender']
edu = edu.rename(columns={'R0000100':'id', 'T9900000': 'grade'})
birth = pd.merge(birth, edu[['id', 'grade']], on='id', how='left')

In [4]:
# Create a range of age values from 21 to 60
age_values = np.arange(20, 61)

# Repeat the rows for each age value
num_repeats = len(age_values)
sample = birth.loc[birth.index.repeat(num_repeats)].reset_index(drop=True)

# Assign the age values to the repeated DataFrame
sample['age'] = np.tile(age_values, len(birth))

In [5]:
marstat.columns = ['id'] + list(range(1979,1995)) + list(range(1996,2019,2))
# deal with negative values
marstat = marstat.replace(-4, 0)
marstat[marstat<0] = np.nan

In [6]:
# convert categorical information to dummies
# if marital status is 0 (never married), 3 (divorced), or 6 (widowed), set the dummy to 0
columns_to_replace = marstat.columns[1:]  # Exclude the first column
marstat[columns_to_replace] = np.where((marstat[columns_to_replace] == 0) |
                                       (marstat[columns_to_replace] == 3) |
                                       (marstat[columns_to_replace] == 6), 0,
                                       marstat[columns_to_replace])
marstat[columns_to_replace] = np.where((marstat[columns_to_replace] == 1) |
                                       (marstat[columns_to_replace] == 2) |
                                       (marstat[columns_to_replace] == 5), 1,
                                       marstat[columns_to_replace])

In [7]:
new_cols = list(range(1995,2018,2))
for col in new_cols:
    marstat[col] = np.nan  # Assign nan (or any other initial value) to create an empty column

In [8]:
new_column_order = ['id'] + list(range(1979,2019))
marstat = marstat[new_column_order]

In [9]:
col_interpolate = marstat.columns[1:]
marstat[col_interpolate] = marstat[col_interpolate].interpolate(method='ffill', axis=1)
marstat[col_interpolate] = marstat[col_interpolate].interpolate(method='bfill', axis=1)

In [10]:
# convert a wide dataframe to a long one
marstat_melted = pd.melt(marstat, id_vars=['id'], var_name='year', value_name='marstat')

In [11]:
spouse_edu.columns = ['id'] + list(range(1979,1995)) + list(range(1996,2019,2))
for col in new_cols:
    spouse_edu[col] = np.nan  # Assign nan (or any other initial value) to create an empty column

In [12]:
spouse_edu = spouse_edu[new_column_order]

In [13]:
# deal with negative values
spouse_edu[spouse_edu<0] = np.nan

In [14]:
col_interpolate = spouse_edu.columns[1:]
spouse_edu[col_interpolate] = spouse_edu[col_interpolate].interpolate(method='ffill', axis=1)
spouse_edu[col_interpolate] = spouse_edu[col_interpolate].interpolate(method='bfill', axis=1)

In [15]:
spouse_edu_melted = pd.melt(spouse_edu, id_vars=['id'], var_name='year', value_name='spouse_grade')

In [16]:
# add birth year to calculate age
spouse_edu_melted = pd.merge(spouse_edu_melted, birth[['id', 'birth_year']], on='id', how='left')
spouse_edu_melted['year'] = spouse_edu_melted['year'].astype(int)
spouse_edu_melted['age'] = spouse_edu_melted['year'] - 1900 - spouse_edu_melted['birth_year']

In [17]:
# add spouse grade to the sample
sample = pd.merge(sample, spouse_edu_melted[['id', 'age', 'spouse_grade']],
                  on=['id', 'age'], how='left')

In [18]:
# add birth year to calculate age
marstat_melted = pd.merge(marstat_melted, birth[['id', 'birth_year']], on='id', how='left')
marstat_melted['year'] = marstat_melted['year'].astype(int)
marstat_melted['age'] = marstat_melted['year'] - 1900 - spouse_edu_melted['birth_year']

In [19]:
# add marital status to the sample
sample = pd.merge(sample, marstat_melted[['id', 'age', 'marstat']],
                  on=['id', 'age'], how='left')

In [20]:
sample['marstat'] = sample.groupby('id')['marstat'].transform(lambda x: x.bfill())
sample['spouse_grade'] = sample.groupby('id')['spouse_grade'].transform(lambda x: x.bfill())

In [21]:
sample['spouse_grade'] = np.where(sample['marstat']==0, np.nan, sample['spouse_grade'])

In [22]:
df = sample[(sample['marstat'] == 1) & (sample['spouse_grade'].isna())]

In [23]:
# Merge df1 and df2 with indicator=True
merged = sample.merge(df['id'], on='id', how='left', indicator=True)

# Filter rows where '_merge' is 'left_only' (meaning they exist in df1 but not in df2)
sample = merged[merged['_merge'] == 'left_only'].drop(columns='_merge')

In [24]:
df = sample[(sample['age'] == 20) & (sample['marstat'] == 1)]

In [25]:
# Merge df1 and df2 with indicator=True
merged = sample.merge(df['id'], on='id', how='left', indicator=True)

# Filter rows where '_merge' is 'left_only' (meaning they exist in df1 but not in df2)
sample = merged[merged['_merge'] == 'left_only'].drop(columns='_merge')

In [26]:
sample['mardur'] = 0

In [27]:
sample = sample[sample['grade'] >= 0]

In [28]:
sample.describe()

,id,birth_year,gender,grade,age,spouse_grade,marstat,mardur
count,416109.000000,416109.000000,416109.000000,416109.000000,416109.000000,195225.000000,389803.00000,416109.0
mean,6241.328702,60.481525,1.452458,13.368115,40.000000,13.453267,0.50083,0.0
std,3643.501369,2.213100,0.497735,2.574056,11.832174,3.316483,0.50000,0.0
min,1.000000,57.000000,1.000000,0.000000,20.000000,0.000000,0.00000,0.0
25%,3128.000000,59.000000,1.000000,12.000000,30.000000,12.000000,0.00000,0.0
50%,6237.000000,60.000000,1.000000,12.000000,40.000000,12.000000,1.00000,0.0
75%,9340.000000,62.000000,2.000000,15.000000,50.000000,16.000000,1.00000,0.0
max,12685.000000,64.000000,2.000000,20.000000,60.000000,99.000000,1.00000,0.0


In [29]:
# Create a new column 'edu' based on 'grade'
def assign_edu(grade):
    if grade < 12:
        return 0
    elif grade == 12:
        return 1
    elif grade < 16:
        return 2
    elif grade == 16:
        return 3
    else:
        return 4

sample['edu'] = sample['grade'].apply(assign_edu)

In [30]:
female = sample[sample['gender'] == 2]
male = sample[sample['gender'] == 1]

In [31]:
# Group by 'age' and count the occurrences of 'marstat' values
equilibrium_f = female.groupby(['age', 'edu'])['marstat'].value_counts().unstack(fill_value=0)

# Rename the columns for clarity
equilibrium_f.columns = ['single_f', 'married_f']
equilibrium_f = equilibrium_f.reset_index()

In [32]:
# Group by 'age' and count the occurrences of 'marstat' values
equilibrium_m = male.groupby(['age', 'edu'])['marstat'].value_counts().unstack(fill_value=0)

# Rename the columns for clarity
equilibrium_m.columns = ['single_m', 'married_m']
equilibrium_m = equilibrium_m.reset_index()

In [33]:
equilibrium = pd.merge(equilibrium_f, equilibrium_m, on=['age', 'edu'], how='left')

In [38]:
sample['birth_year'].describe()

count    416109.000000
mean         60.481525
std           2.213100
min          57.000000
25%          59.000000
50%          60.000000
75%          62.000000
max          64.000000
Name: birth_year, dtype: float64

In [36]:
sample.to_csv('sample79.csv',index=False)
equilibrium.to_csv('equilibrium.csv',index=False)